In [2]:
from math import sin, cos,asin, degrees,sqrt, atan2, radians
import json
import pandas as pd
import numpy as np
import csv
from os import listdir
from os.path import isfile, join
import gc

In [25]:
PATH = '../../../BICIMAD/viajes2017-2019'
DISTANCIA = 300
estaciones_bicimad_totales = pd.read_csv('estaciones_bicimad_totales.csv', sep=";", header=0)

## 4.4.2 Búsqueda de itinerarios

In [36]:
def get_movimientos(df,df_estaciones_bicis):
    list_aux = []
    for index, row in df_estaciones_bicis.iterrows():
        res = df[(df['idplug_station'] == row.id_bicimad_origen) & (df['idunplug_station'] == row.id_bicimad_destino)]
        if not res.empty:
            for index2, row2 in res.iterrows():
                list_aux.append([row2.user_day_code,row2.user_type,row2.travel_time,row2.idunplug_station,row2.ageRange,row2.idplug_station,row2.unplug_hourTime['$date'],row.id_line,row.id_header_origen,row.id_header_destino,'AtoB'])

        res = df[(df['idplug_station'] == row.id_bicimad_destino) & (df['idunplug_station'] == row.id_bicimad_origen)]
        if not res.empty:
            for index2, row2 in res.iterrows():
                list_aux.append([row2.user_day_code,row2.user_type,row2.travel_time,row2.idunplug_station,row2.ageRange,row2.idplug_station,row2.unplug_hourTime['$date'],row.id_line,row.id_header_origen,row.id_header_destino,'BtoA'])
    return pd.DataFrame(list_aux, columns = ['user_day_code','user_type','travel_time','idunplug_station','ageRange','idplug_station','date','id_line','id_header_origen','id_header_destino','direction'])  

## 4.4.1 Limpieza de los conjunto de datos de movimientos

In [37]:
for f in listdir(PATH):
    f_csv = str(DISTANCIA) + "-" + f
    file_json = join(PATH, f)
    file_csv = join(PATH, f_csv.replace('.json','.csv'))
    if isfile(file_json) and ".json" in file_json:
        viajes_mes_df = pd.read_json(file_json,encoding = "ISO-8859-1", lines=True)
        viajes_mes_df = viajes_mes_df.loc[viajes_mes_df['user_type'].isin(['1','2'])]
        get_movimientos(viajes_mes_df,estaciones_bicimad_totales).to_csv(file_csv, sep=';', encoding='utf-8',index=False)
        print(file_json)
        del viajes_mes_df
        viajes_mes_df = ''
        gc.collect()     

../../../BICIMAD/viajes2017-2019\201704_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201705_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201706_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201707_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201708_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201709_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201710_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201711_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201712_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201801_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201802_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201803_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201804_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201805_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201806_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201807_Usage_Bicimad.json
../../../BICIMAD/viajes2017-2019\201808_Usage_Bicimad.js

##### Unimos los ficheros resultantes en un solo archivo CSV

In [50]:
aux = []
for f in listdir(PATH):
    file_json = join(PATH, f)
    if isfile(file_json) and "csv" in file_json and "csv" in file_json:
        aux.append(pd.read_csv(file_json, sep=";", header=0))
pd.DataFrame(pd.concat(aux)).to_csv(PATH + '/' + 'MOVIMIENTOS-GENERAL.csv', sep=';', encoding='utf-8',index=False)

##### Total de movimientos

In [3]:
movimientos = pd.read_csv('movimientos_general.csv', sep=";", header=0)

In [4]:
movimientos.count()[0]

20891

## 4.4.3 Análisis de los resultados

##### Número de lineas encontradas en los movimientos

In [5]:
movimientos.groupby('id_line').groups.keys()

dict_keys([3, 19, 26, 43, 46, 61, 68, 69, 156, 215, 361, 362, 601])

##### Número de lineas encontradas en las rutas 

In [6]:
estaciones_bicimad_totales = pd.read_csv('estaciones_bicimad_totales.csv', sep=";", header=0)

In [8]:
estaciones_bicimad_totales.groupby('id_line').groups.keys()

dict_keys([1, 3, 12, 19, 26, 27, 37, 43, 46, 56, 61, 68, 69, 148, 149, 152, 156, 215, 361, 362, 601])

## 4.4.4 Estaciones de BiciMAD extra

##### Estaciones de bicimad cerca de la Estación de Atocha

In [9]:
estaciones_bicimad_totales[estaciones_bicimad_totales['name_header_origen'].str.contains("Atocha")].groupby('id_bicimad_origen').groups.keys()

dict_keys([84, 85])

In [12]:
estaciones_bicimad_totales_agregado = pd.read_csv('estaciones_bicimad_totales_agregado.csv', sep=";", header=0)

##### Número de rutas encontradas

In [13]:
estaciones_bicimad_totales_agregado.count()[0]

165

In [14]:
movimientos_agregado = pd.read_csv('movimientos_general_agregado.csv', sep=";", header=0)

##### Número de viajes efectuados

In [15]:
movimientos_agregado.count()[0]

26737

##### Número de lineas encontradas en las rutas 

In [16]:
movimientos_agregado.groupby('id_line').groups.keys()

dict_keys([3, 19, 26, 43, 46, 61, 68, 69, 156, 215, 361, 362, 601])